## Expenses on car rental

This analysis aims to investigate expenses on car rental during the current term. Previous analysis I did using Excel shows 1) some politicians systematically spends above the monthly limit of R$ 10K, and 2) some congresspersons rent more than one vehicle every month, which brings certain suspicion: considering they work in DF, are those cars rented outside DF being used by someone else? 

~~**First step:** get a list of congresspersons, the amount reimbursed by them since Jan. 2015 and the dates of those reimbursements. Then we cross these data with the list of companies that rented those vehicles so we can get information on where those rentals occurred.~~ *Done!*

**Second step:** get expenses on toll to check whether those cars left the origin city or not. *In development.*

-- Rodolfo Viana

In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv('../data/2017-06-04-reimbursements.xz',
                      dtype={'applicant_id': np.str,
                             'cnpj_cpf': np.str,
                             'congressperson_id': np.str,
                             'congressperson_name': np.str,
                             'subquota_number': np.str,
                             'issue_date': np.str,
                             'document_id': np.str},
                      low_memory=False)

#### Selecting term, subquota

There are 14,263 reimbursements since Jan. 2015. They sum up to R$ 60,373,960.80.

In [2]:
data = data[data['year'] >= 2015]
data['cnpj_cpf'] = data['cnpj_cpf'].str.replace(r'[\.\/\-]', '')
data_car = data[data['subquota_description'] == 'Automotive vehicle renting or charter']
data_car.subquota_description.value_counts()

Automotive vehicle renting or charter    14263
Name: subquota_description, dtype: int64

In [3]:
data_car.net_values.sum()

60373960.80000005

In [4]:
congressperson_list_car = data_car[['congressperson_name', 
                            'congressperson_id', 
                            'net_values', 
                            'month', 
                            'year', 
                            'issue_date', 
                            'document_id',
                            'cnpj_cpf']]

In [5]:
congressperson_expenses_car = congressperson_list_car.groupby(['congressperson_name', 
                                                               'year', 
                                                               'month', 
                                                               'issue_date', 
                                                               'document_id']).agg({'net_values':sum})

congressperson_expenses_car.head(20)

net_values
congressperson_name year month issue_date            document_id            
ABEL MESQUITA JR.   2015 2     2015-02-02 00:00:00.0 5601321           900.0
                         9     2015-10-26 00:00:00.0 5830987          9307.0
                         11    2015-11-30 00:00:00.0 5862624          9693.9
                         12    2015-12-23 00:00:00.0 5886420          9440.0
                    2016 2     2016-02-29T00:00:00   5929072          7080.0
                         3     2016-03-30T00:00:00   5959422          9900.0
                         4     2016-04-28T00:00:00   5986290          9100.0
                         5     2016-05-31T00:00:00   6011416          9500.0
                         6     2016-06-30T00:00:00   6041818          9600.0
                         7     2016-07-29T00:00:00   6069097          7800.0
                         8     2016-08-30T00:00:00   6097664          8520.0
                         9     2016-09-30T00:00:00   6139753          6920.0
                         10    2016-10-31T00:00:00   6151586          7000.0
                         11    2016-11-29T00:00:00   6164350          8320.0
                    2017 1     2017-01-30T00:00:00   6197024          7720.0
                         2     2017-02-22T00:00:00   6224629          7920.0
                         3     2017-03-30T00:00:00   6265325          9890.0
                         4     2017-04-28T00:00:00   6272909          9477.2
ADAIL CARNEIRO      2015 2     2015-02-14 00:00:00.0 5612808          9980.0
                         3     2015-03-06 00:00:00.0 5640977          9980.0

#### Getting companies dataset, excluding those from DF, merging with reimbursements dataset

There are 10,945 companies outside DF. The receipts sum up to R$ 49,156,001.56.

In [6]:
companies = pd.read_csv('../data/2017-05-21-companies-no-geolocation.xz', low_memory=False)
companies['cnpj'] = companies['cnpj'].str.replace(r'[\.\/\-]', '')
companies_cars = companies[companies['state'] != 'DF']

In [7]:
dataset_car = pd.merge(data_car, companies_cars, how='outer',
                   left_on='cnpj_cpf', right_on='cnpj')

In [8]:
congressperson_expenses_dataset_car = dataset_car.groupby(['congressperson_name',
                                                    'applicant_id',
                                                    'year', 
                                                    'month', 
                                                    'issue_date',
                                                    'cnpj',
                                                    'name',
                                                    'city',
                                                    'state_y',
                                                    'document_id']).agg({'net_values':sum})
full_report_car = congressperson_expenses_dataset_car.reset_index()

In [9]:
full_report_car.name.count()

10945

In [10]:
full_report_car.net_values.sum()

49156001.55999989

#### Getting outliers

#### On x times `std value` and threshold

As I am not familiar with statistics, I tested 2 x `std value` and 3 x `std value` to get the threshold. I took CNPJs with more than 20 receipts -- which is a good sample and also shows frequency on hiring the companies' services -- for 1 or more congresspersons -- for we are analyzing companies in different states, and preliminary analysis (via Excel) shows a company is hired most of the time for a single congressperson.

The first scenario (2 x `std value`) shows we have 99 outliers. The second one (3 x `std value`) gives us 22 outliers.

In [11]:
full_report_car.net_values.describe()

count    10945.000000
mean      4491.183331
std       2694.178468
min          3.230000
25%       2492.100000
50%       3900.000000
75%       6200.000000
max      10900.000000
Name: net_values, dtype: float64

In [12]:
from scipy.stats import normaltest

def normaltest_pvalue(values):
    if len(values) >= 20:
        return normaltest(values).pvalue
    else:
        return 1

net_values_by_cnpj_1 = full_report_car.groupby('cnpj')['net_values'] \
    .agg([len, np.mean, np.std, normaltest_pvalue]) \
    .sort_values('len', ascending=False) \
    .reset_index()
net_values_by_cnpj_1['threshold'] = net_values_by_cnpj_1['mean'] + \
    2 * net_values_by_cnpj_1['std']
applicants_per_cnpj = full_report_car.groupby('cnpj')['applicant_id'] \
    .aggregate(lambda x: len(set(x))).reset_index() \
    .rename(columns={'applicant_id': 'congresspeople'})
net_values_by_cnpj_1 = pd.merge(net_values_by_cnpj_1, applicants_per_cnpj)
net_values_by_cnpj_1.head()

,cnpj,len,mean,std,normaltest_pvalue,threshold,congresspeople
0,07976147002295,293.0,2373.325290,1095.545391,7.037273e-01,4564.416071,48
1,21652672000151,188.0,3205.250479,862.410949,2.241561e-11,4930.072376,7
2,02570474000102,169.0,5007.603550,1535.344025,9.963501e-01,8078.291601,4
3,08659172000183,152.0,4128.017237,1503.747136,9.874299e-02,7135.511509,5
4,09400977000170,145.0,6572.993793,2472.293774,6.448993e-04,11517.581342,5


In [13]:
len(net_values_by_cnpj_1.query('normaltest_pvalue < .05')) / len(net_values_by_cnpj_1)

0.2504012841091493

In [14]:
def normaltest_pvalue(values):
    if len(values) >= 20:
        return normaltest(values).pvalue
    else:
        return 1

net_values_by_cnpj_2 = full_report_car.groupby('cnpj')['net_values'] \
    .agg([len, np.mean, np.std, normaltest_pvalue]) \
    .sort_values('len', ascending=False) \
    .reset_index()
net_values_by_cnpj_2['threshold'] = net_values_by_cnpj_2['mean'] + \
    3 * net_values_by_cnpj_2['std']
applicants_per_cnpj = full_report_car.groupby('cnpj')['applicant_id'] \
    .aggregate(lambda x: len(set(x))).reset_index() \
    .rename(columns={'applicant_id': 'congresspeople'})
net_values_by_cnpj_2 = pd.merge(net_values_by_cnpj_2, applicants_per_cnpj)
net_values_by_cnpj_2.head()

,cnpj,len,mean,std,normaltest_pvalue,threshold,congresspeople
0,07976147002295,293.0,2373.325290,1095.545391,7.037273e-01,5659.961462,48
1,21652672000151,188.0,3205.250479,862.410949,2.241561e-11,5792.483325,7
2,02570474000102,169.0,5007.603550,1535.344025,9.963501e-01,9613.635627,4
3,08659172000183,152.0,4128.017237,1503.747136,9.874299e-02,8639.258645,5
4,09400977000170,145.0,6572.993793,2472.293774,6.448993e-04,13989.875116,5


In [15]:
len(net_values_by_cnpj_2.query('normaltest_pvalue < .05')) / len(net_values_by_cnpj_2)

0.2504012841091493

In [16]:
data_with_threshold_1 = pd.merge(full_report_car, net_values_by_cnpj_1, on='cnpj') \
    .sort_values('net_values', ascending=False)

In [17]:
data_with_threshold_2 = pd.merge(full_report_car, net_values_by_cnpj_2, on='cnpj') \
    .sort_values('net_values', ascending=False)

In [18]:
outliers_1 = data_with_threshold_1.query('(congresspeople >= 1) & (len >= 20) & (net_values > threshold)')
print(len(outliers_1), outliers_1['net_values'].sum())

99 650909.4999999999


In [19]:
outliers_2 = data_with_threshold_2.query('(congresspeople >= 1) & (len >= 20) & (net_values > threshold)')
print(len(outliers_2), outliers_2['net_values'].sum())

22 120224.65


#### Getting toll expenses

In [20]:
companies_toll = companies[companies['main_activity_code'] == '52.21-4-00']

In [21]:
data_toll = data[data['subquota_description'] == 'Taxi, toll and parking']

In [22]:
dataset_toll = pd.merge(data_toll, companies_toll, how='outer',
                   left_on='cnpj_cpf', right_on='cnpj')

In [23]:
congressperson_list_toll = data_toll[['congressperson_name', 
                            'congressperson_id', 
                            'net_values', 
                            'month', 
                            'year', 
                            'issue_date', 
                            'document_id',
                            'cnpj_cpf']]

In [24]:
congressperson_expenses_dataset_toll = dataset_toll.groupby(['congressperson_name',
                                                    'applicant_id',
                                                    'year', 
                                                    'month', 
                                                    'issue_date',
                                                    'cnpj',
                                                    'name',
                                                    'city',
                                                    'state_y',
                                                    'document_id']).agg({'net_values':sum})
full_report_toll = congressperson_expenses_dataset_toll.reset_index()

In [25]:
full_report_toll.name.count()

32114

In [26]:
full_report_toll.net_values.sum()

216930.90000001166

In [27]:
full_report_toll.head()

,congressperson_name,applicant_id,year,month,issue_date,cnpj,name,city,state_y,document_id,net_values
0,ADELMO CARNEIRO LEÃO,2906,2015.0,9.0,2015-09-24 00:00:00.0,09326342000170,AUTOPISTA FERNAO DIAS S.A.,POUSO ALEGRE,MG,5814123,1.6
1,ADELMO CARNEIRO LEÃO,2906,2015.0,9.0,2015-09-24 00:00:00.0,09326342000170,AUTOPISTA FERNAO DIAS S.A.,POUSO ALEGRE,MG,5814129,1.6
2,ADELMO CARNEIRO LEÃO,2906,2015.0,9.0,2015-09-24 00:00:00.0,09326342000170,AUTOPISTA FERNAO DIAS S.A.,POUSO ALEGRE,MG,5814136,1.6
3,ADELMO CARNEIRO LEÃO,2906,2015.0,9.0,2015-09-24 00:00:00.0,09326342000170,AUTOPISTA FERNAO DIAS S.A.,POUSO ALEGRE,MG,5814142,1.6
4,ADELMO CARNEIRO LEÃO,2906,2015.0,9.0,2015-09-24 00:00:00.0,09326342000170,AUTOPISTA FERNAO DIAS S.A.,POUSO ALEGRE,MG,5814145,1.6
